## Upset Prediction Project - Data Wrangling

### Integration of player-level data

For my upset prediction project, one of my goals was to obtain data that allowed me to create additional, unique features that could not be created from team-level data alone. Using the Python package BeautifulSoup, I wrote a script to scrape over 3,000 tables with player-level data from two separate sports statistics websites, www.espn.com and www.sports-reference.com. 

This notebook describes the process I used to clean and integrate this player-level data. 

In [1]:
# importing packages for wrangling tasks
import pandas as pd
import numpy as np
import string

# import package for fuzzy string matching
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# allow wide display of columns    
pd.set_option('display.max_columns', None)

In [2]:
# create a function to quickly tabulate a dataframe column
def tab(dfcol):
    t = pd.crosstab(index=dfcol, columns="count")
    print t
    print t/t.sum()

#### Fuzzy matching player names
I had obtained three separate sets of player-level data. Prior to computing team features, I need to link these files using player identifiers.

I start by importing the separate data files:

In [3]:
# import the .csv files as pandas dataframes
data_in = "../data/ncaa/"
pstats = pd.read_csv(data_in + 'player_stats.csv')
ros = pd.read_csv(data_in + 'rosters.csv')
espn = pd.read_csv(data_in + 'espn_tabs.csv')

Each dataframe contains unique information about college basketball players. As an example, I show a few rows of each dataframe for a single team, the 2012 Kentucky Wildcats. 

In [4]:
ex_season = 2012
sr_team = 'kentucky'
espn_team = 'Kentucky Wildcats'

The player stats dataframe has box-score statistics for each player. 

In [5]:
# show pstats data
pstats[(pstats['Team'] == sr_team) & (pstats['Season'] == ex_season)].head()

,2P,2P%,2PA,3P,3P%,3PA,AST,BLK,FG,FG%,FGA,FT,FT%,FTA,G,MP,PF,PTS,Player,STL,Season,TOV,TRB,Team
8857,5.2,0.653,7.9,0.1,0.150,0.5,1.3,4.7,5.3,0.623,8.4,3.6,0.709,5.1,40.0,32.0,2.0,14.2,Anthony Davis,1.4,2012,1.0,10.4,kentucky
8858,2.5,0.481,5.2,1.9,0.466,4.1,1.5,0.1,4.4,0.474,9.2,3.1,0.826,3.7,40.0,31.2,1.8,13.7,Doron Lamb,0.5,2012,1.1,2.7,kentucky
8859,4.2,0.528,8.0,0.4,0.327,1.3,1.3,1.8,4.7,0.500,9.3,2.6,0.627,4.2,38.0,29.3,2.4,12.3,Terrence Jones,1.3,2012,1.6,7.2,kentucky
8860,3.7,0.535,6.9,0.3,0.255,1.3,1.9,0.9,4.0,0.491,8.2,3.6,0.745,4.8,40.0,31.1,2.4,11.9,Michael Kidd-Gilchrist,1.0,2012,2.2,7.4,kentucky
8861,3.0,0.438,6.8,0.7,0.325,2.0,4.8,0.3,3.6,0.412,8.8,2.1,0.714,3.0,40.0,32.6,2.3,10.0,Marquis Teague,0.9,2012,2.7,2.5,kentucky


The roster dataframe has details about each player's year in school, size, and position. 

In [6]:
ros[(ros['Team'] == sr_team) & (ros['Season'] == ex_season)].head()

,Class,Height,Player,Pos,Season,Team
8857,JR,6-5,Twany Beckham,G,2012,kentucky
8858,FR,6-10,Anthony Davis,F,2012,kentucky
8859,SO,6-9,Terrence Jones,F,2012,kentucky
8860,FR,6-7,Michael Kidd-Gilchrist,F,2012,kentucky
8861,SO,6-4,Doron Lamb,G,2012,kentucky


The espn dataframe has box-score statistics, similar to the player stats dataframe above.

In [7]:
espn[(espn['Team'] == espn_team) & (espn['Season'] == ex_season)].head()

,Player,GP,MIN,PPG,RPG,APG,SPG,BPG,TPG,FG%,FT%,3P%,Team,espn_id,Season
3746,Anthony Davis,40,32.0,14.2,10.4,1.3,1.4,4.7,1.0,0.623,0.709,0.150,Kentucky Wildcats,96,2012
3747,Doron Lamb,40,31.2,13.7,2.7,1.5,0.5,0.1,1.1,0.474,0.826,0.466,Kentucky Wildcats,96,2012
3748,Terrence Jones,38,29.3,12.3,7.2,1.3,1.3,1.8,1.6,0.500,0.627,0.327,Kentucky Wildcats,96,2012
3749,Michael Kidd-Gilchrist,40,31.1,11.9,7.4,1.9,1.0,0.9,2.2,0.491,0.745,0.255,Kentucky Wildcats,96,2012
3750,Marquis Teague,40,32.6,10.0,2.5,4.8,0.9,0.3,2.7,0.412,0.714,0.325,Kentucky Wildcats,96,2012


But for some statistics, the espn dataframe is more complete, especially average minutes per game. 

In [8]:
# select only rows with season above 2002, the minimum value for espn data
# count nonmissing values for average minutes per game from each file
print 'Valid rows for minutes column for player stats'
print pstats[pstats['Season'] >= 2002].describe().loc[:, 'MP'].loc['count']
print 'Valid rows for minutes column for espn'
print espn.describe().loc[:, 'MIN'].loc['count']

Valid rows for minutes column for player stats
7879.0
Valid rows for minutes column for espn
14419.0


Because I intend to use player average minutes per game to compute features, and also player size and position, I need to clean and integrate all of this data together. 

No team should have the same player listed twice for the same year, so I need to verify this and remove any duplicates. 

In [9]:
# function to return whether any player names duplicated for unique pairs of season and team
def any_duplicates(df):
    df_dup = df[df[['Player', 'Season', 'Team']].duplicated()]
    return df_dup.shape[0] != 0

print any_duplicates(pstats)
print any_duplicates(ros)
print any_duplicates(espn)

True
True
True


There are some duplicated players in each dataframe. I'll remove the duplicated rows before I start merging the data together. 

In [10]:
# drop duplicate rows
dup_cols = ['Player', 'Team', 'Season']
pstats = pstats[~pstats[dup_cols].duplicated()]
ros = ros[~ros[dup_cols].duplicated()]
espn = espn[~espn[dup_cols].duplicated()]

In [11]:
# verify no duplicates
map(lambda x: any_duplicates(x), [pstats, ros, espn])

[False, False, False]

I'm now ready to start merging the player dataframes together.  
Because two of the dataframes come from the same source, the player names are identical, which means I can merge them easily by joining on the "Player", "Team", and "Season" columns. 

In [12]:
# merge the two sportsreference files
sr = pd.merge(pstats, ros, how='inner',
              on=['Player', 'Team', 'Season'])

In [13]:
# show 5 rows to illustrate result of inner join
sr[(sr['Team'] == sr_team) & (sr['Season'] == ex_season)].head()

,2P,2P%,2PA,3P,3P%,3PA,AST,BLK,FG,FG%,FGA,FT,FT%,FTA,G,MP,PF,PTS,Player,STL,Season,TOV,TRB,Team,Class,Height,Pos
8857,5.2,0.653,7.9,0.1,0.150,0.5,1.3,4.7,5.3,0.623,8.4,3.6,0.709,5.1,40.0,32.0,2.0,14.2,Anthony Davis,1.4,2012,1.0,10.4,kentucky,FR,6-10,F
8858,2.5,0.481,5.2,1.9,0.466,4.1,1.5,0.1,4.4,0.474,9.2,3.1,0.826,3.7,40.0,31.2,1.8,13.7,Doron Lamb,0.5,2012,1.1,2.7,kentucky,SO,6-4,G
8859,4.2,0.528,8.0,0.4,0.327,1.3,1.3,1.8,4.7,0.500,9.3,2.6,0.627,4.2,38.0,29.3,2.4,12.3,Terrence Jones,1.3,2012,1.6,7.2,kentucky,SO,6-9,F
8860,3.7,0.535,6.9,0.3,0.255,1.3,1.9,0.9,4.0,0.491,8.2,3.6,0.745,4.8,40.0,31.1,2.4,11.9,Michael Kidd-Gilchrist,1.0,2012,2.2,7.4,kentucky,FR,6-7,F
8861,3.0,0.438,6.8,0.7,0.325,2.0,4.8,0.3,3.6,0.412,8.8,2.1,0.714,3.0,40.0,32.6,2.3,10.0,Marquis Teague,0.9,2012,2.7,2.5,kentucky,FR,6-2,G


The player statistics are now paired with the other player variables. 

The task of merging this data with the espn data is more complicated, because of differences in the team names. In the example above, the same team is named "kentucky" in the sportsreference data, and "Kentucky Wildcats" in the espn data. 
To address this problem I previously created a data table with all of the unique identifiers for each team, shown below. 

In [14]:
teams_seasons = pd.read_csv(data_in + 'teams_seasons.csv')
team_match = pd.read_csv(data_in + 'team_match.csv')
team_info = pd.merge(teams_seasons, team_match,
                     how='inner', on='team_id')
team_info.head()

,season,team_id,srname,espn_id,match
0,1985,1116,arkansas,8.0,match
1,1989,1116,arkansas,8.0,match
2,1990,1116,arkansas,8.0,match
3,1991,1116,arkansas,8.0,match
4,1992,1116,arkansas,8.0,match


The team_info dataframe has an unique ID number, the team name in the sportsreference data, and a unique numeric identifier from the espn data. This data gives me what I need to link teams in one file to another. 

I first need to merge the team_info data into the player dataframes.

In [15]:
# merge team identifier data with player dataframes
sr_ti = pd.merge(sr, team_info, how='inner', left_on=['Team', 'Season'],
                 right_on=['srname', 'season'])
espn_ti = pd.merge(espn, team_info, how='inner',
                   left_on=['espn_id', 'Season'],
                   right_on=['espn_id', 'season'])

To simplify the data I also keep only the columns I need to merge players with each other. That includes the player name, season, and the two team identifiers. 

In [16]:
# keep only columns I need to match players
sr_players = sr_ti.loc[:, ['Player', 'Season', 'team_id']]
# keep only years after 2001 to match with espn data
sr_players = sr_players[sr_players['Season'] > 2001]
espn_players = espn_ti.loc[:, ['Player', 'Season', 'team_id']]

In [17]:
print sr_players.shape
print espn_players.shape

(13497, 3)
(13543, 3)


As a first step in matching the player data, I use an outer join that attempts to join the data on player names, and keeps any unmerged rows from either dataframe. 

In [18]:
mrg_players = pd.merge(sr_players, espn_players, how='outer',
                       on=['Player', 'Season', 'team_id'], indicator=True)
# use tab function to inspect number of successful joins
tab(mrg_players['_merge'])

col_0       count
_merge           
left_only     547
right_only    593
both        12950
col_0          count
_merge              
left_only   0.038822
right_only  0.042087
both        0.919092


This merge left around 8% of names unmatched. I needed to separate the nonmatched names, determine why they didn't match, and figure out a way to link them together.  

To see what causes the match failures, I examine a sample of 10 rows. 

In [19]:
nomatch = mrg_players[mrg_players['_merge'] != "both"].copy()
nomatch = nomatch.sort_values(['team_id', 'Season'])
nomatch.head(10)

,Player,Season,team_id,_merge
31,Brett Mcknight,2009,1103,left_only
13497,Brett McKnight,2009,1103,right_only
46,Brett Mcknight,2011,1103,left_only
13498,Brett McKnight,2011,1103,right_only
68,Adedeji Ibitayo,2013,1103,left_only
13499,Deji Ibitayo,2013,1103,right_only
76,Mo Williams,2002,1104,left_only
13500,Maurice Williams,2002,1104,right_only
11537,Mo Williams,2003,1104,left_only
13979,Maurice Williams,2003,1104,right_only


The same players are present in both files, but they have variations in the name due to punctuation, spelling, or use of nicknames. 

Knowing this, I realize I can probably match more names with the inner merge above simply by re-formatting the names. 

In [20]:
print ' craig carey'
print ' craig carey'.strip()

 craig carey
craig carey


In [21]:
# create function to format player name
def clean_name(x):
    x = x.lower() # remove all upper case
    x = x.strip() # remove leading and trailing white space
    x = x.translate(None, string.punctuation) # remove non-alphabetic punctuation
    x = x.replace(' jr', '') # remove suffixes
    x = x.replace(' iii', '')
    x = x.replace(' ii', '')
    x = x.replace(' iv', '')
    return x

In [22]:
# create a new "name" column with the cleaned player name
sr_players['name'] = sr_players['Player'].apply(clean_name)
espn_players['name'] = espn_players['Player'].apply(clean_name)

In [23]:
# repeat inner merge, using cleaned name
mrg_players = pd.merge(sr_players, espn_players, how='outer',
                       on=['name', 'Season', 'team_id'], indicator=True)
# use tab function to inspect number of successful joins
tab(mrg_players['_merge'])

col_0       count
_merge           
left_only     295
right_only    341
both        13202
col_0          count
_merge              
left_only   0.021318
right_only  0.024642
both        0.954040


Now around 2% of the names remain unmatched.  

In [24]:
nomatch = mrg_players[mrg_players['_merge'] != "both"].copy()
nomatch = nomatch.sort_values(['name', 'Season'])
nomatch.head(10)

,Player_x,Season,team_id,name,Player_y,_merge
13677,NaN,2012,1361,aaron douglas,Aaron Douglas,right_only
13678,NaN,2013,1361,aaron douglas,Aaron Douglas,right_only
7716,Aaron Douglass,2012,1361,aaron douglass,NaN,left_only
7727,Aaron Douglass,2013,1361,aaron douglass,NaN,left_only
13532,NaN,2008,1165,aaron osgood,Aaron Osgood,right_only
1657,Aaron Richardson-Osgood,2008,1165,aaron richardsonosgood,NaN,left_only
13775,NaN,2016,1355,adam dykman,Adam Dykman,right_only
13711,NaN,2004,1416,adam gill,Adam Gill,right_only
68,Adedeji Ibitayo,2013,1103,adedeji ibitayo,NaN,left_only
10754,Ahmed Hamdy Mohamed,2016,1433,ahmed hamdy mohamed,NaN,left_only


To match the remaining names, I used the python package fuzzywuzzy. The extractOne function  compares a given string to a list of strings supplied to the function, and extracts one "best match" from the list. The function returns a tuple of the best match and a score that reflects the accuracy of the match. 

Here I show an example using names from the table above. I pass it a string to be matched from the sr dataframe ('aaron douglass'), and all the unmatched names from the espn data. 

In [25]:
names_ro = nomatch[nomatch['_merge'] == 'right_only']['name'].values
process.extractOne('aaron douglass', names_ro)

('aaron douglas', 96)

The extractOne function pulls the correct name (I'm 99% confident aaron douglas and aaron douglass are the same person). 

However, using the function in this default manner compares a single name to over 400 names. Because both sources of data contain the player's team, I can tailor this function to be more efficient and accurate for my purposes. 

I created a function to identify the player's team and season from the "left_only" rows, and compare the player's name only to the list of unmatched names from the corresponding team and season from the "right_only" rows.  

In [26]:
# create dataframe of non-matched player names, separately for each source
nomatch_sr = nomatch[nomatch._merge == "left_only"].copy()
nomatch_sr.drop('_merge', inplace=True, axis=1)
nomatch_espn = nomatch[nomatch._merge == "right_only"].copy()
nomatch_espn.drop('_merge', inplace=True, axis=1)

# group by team and season, create dictionary of non-matched espn names to use in the function
e = nomatch_espn.groupby(['team_id','Season'])['Player_y']
espn_dict = dict(list(e))

In [27]:
# write the function to selectively match using the player's team and season
#plist = []
def match_name_team(row):
    try:
        name = row['Player_x']
        team_id = row['team_id']
        season = row['Season']
        espn_list = espn_dict.get((team_id, season)).values
        name_mat, score = process.extractOne(name, espn_list)
        player_list =  [name, team_id, season, name_mat, score]
        #plist.append(list)
        return player_list
    except:
        return [name, team_id, season, '', 0]

# apply the function to the nonmatched sportsreference player dataframe
matches = nomatch_sr.apply(match_name_team, axis=1).tolist()
columns = ['Player_x', 'team_id', 'Season', 'Player_y', 'score']
df = pd.DataFrame(matches, columns=columns)

After running the function, I can inspect the matched names to assess the quality of the matches. Here are the names of players with the highest scores. 

In [28]:
df.sort_values('score', ascending=False).head(10)

,Player_x,team_id,Season,Player_y,score
21,Baye Moussa Keita,1393,2011,Baye-Moussa Keita,100
131,John Caleb Sanders,1251,2013,John-Caleb Sanders,100
219,Pierre-Marie Altidor Cespedes,1211,2005,Pierre-Marie Altidor-Cespedes,100
163,Marc Eddy Norelia,1195,2016,Marc-Eddy Norelia,100
24,Baye Moussa Keita,1393,2014,Baye-Moussa Keita,100
23,Baye Moussa Keita,1393,2013,Baye-Moussa Keita,100
22,Baye Moussa Keita,1393,2012,Baye-Moussa Keita,100
221,Pierre-Marie Altidor Cespedes,1211,2007,Pierre-Marie Altidor-Cespedes,100
220,Pierre-Marie Altidor Cespedes,1211,2006,Pierre-Marie Altidor-Cespedes,100
275,Trey McKinney-Jones,1274,2013,Trey McKinney Jones,100


These are names with minor differences in spelling or punctuation that are all matched well by the function. 

Here are the names of players who matched with the lowest scores.

In [29]:
# inspect low-scoring matches
#df[df['score'] < 65].sort_values('score', ascending=False)
df[df['score'] != 0].sort_values('score', ascending=True).head(15)

,Player_x,team_id,Season,Player_y,score
264,Terrell White,1172,2002,Puff SummÃ¯ss,8
165,Mark Johnson,1231,2002,Scott May,19
251,Skyler Flatten,1355,2016,Adam Dykman,24
132,John Johnston,1401,2006,Kenneth 'Red' White,26
144,Josh Heath,1274,2016,Chris Stowell,26
25,Beau Brown,1355,2016,Adam Dykman,29
246,Scootie Randall,1396,2009,Semaj Inge,32
280,Walter Pitchford,1196,2012,Erik Jay Murphy,32
227,Randall Jackson,1411,2003,Michael Sneed,36
78,Daylon Guy,1114,2011,Chuck Guy,48


The matches with scores below 50 look highly suspect. Inspecting a few of these revealed players who only existed in one file but not the other due to being on the official roster but not acquiring any playing time. They are true nonmatches, and dropping them will be inconsequential for feature engineering. 

In [30]:
# keep only matches with scores above 50, drop score column
df = df.loc[df.score > 50]
df = df.drop(columns=['score'])

In [31]:
# combine the matches from inner join and fuzzy matches into one dataframe
both = mrg_players[mrg_players['_merge'] == "both"].copy()
both = both.drop(columns=['name', '_merge'])

In [32]:
players_all = pd.concat([both, df], sort=False)
players_all = players_all.rename(columns={'Player_x': 'player_sr',
                                         'Player_y': 'player_espn'})

The result is a dataframe containing each player's original name from both the sportsreference source and the espn source. 

In [33]:
players_all.head()

,player_sr,Season,team_id,player_espn
0,Nick Welch,2004,1102,Nick Welch
1,Tim Keller,2004,1102,Tim Keller
2,Antoine Hood,2004,1102,Antoine Hood
3,A.J. Kuhle,2004,1102,A.J. Kuhle
4,Joel Gerlach,2004,1102,Joel Gerlach


I can test the effectiveness of the matched names by attempting to merge the two sources of data: 

In [34]:
# merge sportsreference file with the player name matches
sr_mrg = pd.merge(sr_players, players_all, how='inner',
                  left_on=['Player', 'team_id', 'Season'],
                  right_on=['player_sr', 'team_id', 'Season'])
mrg2 = pd.merge(sr_mrg, espn_players, how='outer',
               left_on=['player_espn', 'team_id', 'Season'],
               right_on=['Player', 'team_id', 'Season'],
               indicator=True)

In [35]:
tab(mrg2._merge)

col_0       count
_merge           
right_only     75
both        13468
col_0          count
_merge              
right_only  0.005538
both        0.994462


After this merge, 99.4% of the player rows have been linked.  
To finish this portion of my project, I save the paired player names to a data file to use later. 

In [36]:
mrg2.to_csv(data_in + 'player_match.csv')